# POC

In [1]:
%load_ext autoreload
%autoreload 2

# Global imports

In [2]:
import sys

from torch.nn import CrossEntropyLoss
from torch.optim import Adam

# Local imports

In [3]:
sys.path.append('..')

from base_code.models.poc import NeuralNetwork as NN
from base_code.datasets import DryBeansDataset as DBD
from base_code.train_test import train, test

from base_code.preprocessing.nomalize_standarizer import pipeline as normalize_standarize
from base_code.dataloaders.base import ContinualLearningDataLoader as DataLoader

# Data preprocessing

In [16]:
dataset = DBD()

In [29]:
dataset.get_task_range(0, 2).labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [30]:
pipeline = normalize_standarize(dataset.features)

In [31]:
X = pipeline.transform(dataset.features)
dataset.features = X

# DataLoader settings

In [48]:
train_dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

# Neural Netwrok

In [36]:
model = NN(features_shape=dataset.features_shape, output_shape=dataset.output_shape)

In [37]:
loss_fn = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-3)

# Train - Test

## Concurrent

In [50]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(model, train_dataloader, loss_fn, optimizer)
    test(model, train_dataloader, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.633937  [   10/13611]
loss: 1.266453  [ 1010/13611]
loss: 1.165857  [ 2010/13611]
loss: 1.165542  [ 3010/13611]
loss: 1.465427  [ 4010/13611]
loss: 1.265411  [ 5010/13611]
loss: 1.265865  [ 6010/13611]
loss: 1.358413  [ 7010/13611]
loss: 1.265203  [ 8010/13611]
loss: 1.365419  [ 9010/13611]
loss: 1.265438  [10010/13611]
loss: 1.445026  [11010/13611]
loss: 1.234348  [12010/13611]
loss: 1.465557  [13010/13611]
Test Error: 
 Accuracy: 77.4%, Avg loss: 1.390860 

Done!


## Sequential

In [49]:
epochs = 5
for task_id in range(train_dataloader.get_tasks_length()):
    print(f"Task {task_id+1}\n-------------------------------")
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(model, train_dataloader.get_task(task_id), loss_fn, optimizer)
        test(model, train_dataloader.get_task_range(0, task_id+1), loss_fn)

Task 1
-------------------------------
Epoch 1
-------------------------------
loss: 2.135342  [   10/ 2027]
loss: 1.165468  [ 1010/ 2027]
loss: 1.165423  [ 2010/ 2027]
Test Error: 
 Accuracy: 100.0%, Avg loss: 1.166078 

Epoch 2
-------------------------------
loss: 1.165427  [   10/ 2027]
loss: 1.165600  [ 1010/ 2027]
loss: 1.165424  [ 2010/ 2027]
Test Error: 
 Accuracy: 100.0%, Avg loss: 1.165626 

Epoch 3
-------------------------------
loss: 1.165423  [   10/ 2027]
loss: 1.165423  [ 1010/ 2027]
loss: 1.165424  [ 2010/ 2027]
Test Error: 
 Accuracy: 100.0%, Avg loss: 1.165498 

Epoch 4
-------------------------------
loss: 1.165424  [   10/ 2027]
loss: 1.165425  [ 1010/ 2027]
loss: 1.165426  [ 2010/ 2027]
Test Error: 
 Accuracy: 100.0%, Avg loss: 1.165466 

Epoch 5
-------------------------------
loss: 1.165427  [   10/ 2027]
loss: 1.165423  [ 1010/ 2027]
loss: 1.165422  [ 2010/ 2027]
Test Error: 
 Accuracy: 100.0%, Avg loss: 1.165453 

Task 2
-------------------------------
Epoch 1